In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用自定义模型评估导入到Vertex AI模型注册中

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_evaluation%2Fget_started_with_custom_model_evaluation_import.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/get_started_with_custom_model_evaluation_import.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

注意：本笔记本是在以下环境中进行测试的：

- Python 版本 = 3.9

## 概述

本教程展示了如何使用Vertex AI Model Evaluation将自定义模型评估导入到现有的Vertex AI Model Registry条目中。

了解更多关于[Vertex AI中的模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)。

### 目标

在本教程中，您将学习如何构建和上传自定义模型评估，并将自定义模型评估上传到Vertex AI模型注册表中的模型资源条目。

此教程使用以下Google Cloud ML服务和资源：

- Vertex AI模型评估
- Vertex AI模型注册表

执行的步骤包括：

- 导入一个预训练（认可的）模型到Vertex AI模型注册表中。
- 构建一个自定义模型评估。
- 将模型评估指标导入到Vertex AI模型注册表中相应的模型。
- 列出Vertex AI模型注册表中相应模型的模型评估。
- 构建第二个自定义模型评估。
- 将第二个模型评估指标导入到Vertex AI模型注册表中相应的模型。
- 列出相应模型的第二个模型评估。Learn more about [Model Evaluation in Vertex AI](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)。

模型

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

费用

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，和 [云存储定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据预计使用量生成费用估算。

开始吧##

安装Python的Vertex AI SDK和其他必需的包。

In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade google-cloud-aiplatform \
                                tensorflow==2.15.1 \
                                tensorflow-hub

### 重新启动运行时（仅限Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。在继续下一步之前请等待它完成。⚠️</b>
</div>

### 在Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

- *{给笔记本作者的备注：对于任何需要是唯一的用户提供的字符串（比如存储桶名称或模型ID），请在末尾加上“-unique”，以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持 GPU 的容器映像，以及分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个 GPU 容器映像，并且为每个 VM 分配 4 个 Nvidia Telsa T4 GPU，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定`(None, None)`以使用容器映像在 CPU 上运行。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)，以及[GPU 价格](https://cloud.google.com/compute/gpus-pricing)。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为预测设置预构建的Docker容器镜像。

要获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.15.1".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到Vertex AI模型资源。一旦您拥有了一个Vertex AI模型资源，该模型就可以部署到Vertex AI端点资源。

下载预训练模型

首先，您从TensorFlow Hub下载预训练模型。该模型以TF.Keras层的形式下载。为了完成模型，在本例中，您创建了一个带有下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 32, 32, 3])

tfhub_model.summary()

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 上传 TensorFlow Hub 模型到 Vertex AI 模型资源

最后，您可以使用 `upload()` 方法将 TFHub 模型的模型工件上传到 Vertex AI 模型资源，具体参数如下：

- `display_name`：模型资源的人类可读的名称。
- `artifact_uri`：模型包的 Cloud Storage 位置。
- `serving_container_image_uri`：用于服务的容器镜像。

将模型上传到 Vertex AI 模型资源是一个长时间运行的操作，可能需要一些时间

*注意:* 当您将模型工件上传到 Vertex AI 模型资源时，需要指定相应的部署容器镜像。

In [ ]:
model = aiplatform.Model.upload(
    display_name="resnet",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
)

print(model)

## 自定义模型评估简介

在训练自定义模型时，通常要对训练的模型进行一定形式的评估。然后可以使用`import_model_evaluation()` 方法将自定义模型评估导入到对应的模型中的 Vertex AI Model Registry 中。一旦导入，您就可以使用`list_model_evaluations()` 方法检索自定义模型评估。

Vertex AI Model Registry 支持为模型导入多个模型评估，其中每个评估都由唯一的`display_name`进行区分。

### 创建模型评估

首先，您要以与模型评估的预定义模式之一相符的格式创建模型评估。在此示例中，您将使用分类指标的模式，并将以下评估指标子集以字典的形式指定：

- `logLoss`：对数损失。
- `auPrc`：准确性。

然后，使用以下参数构造`ModelEvaluation`对象：

- `display_name`：评估指标的人类可读名称。
- `metrics_schema_uri`：特定类型的评估指标的模式。
- `metrics`：包含评估指标的字典。

了解更多关于[评估指标模式](https://cloud.google.com/vertex-ai/docs/evaluation/introduction#features)。

In [ ]:
metrics = {"logLoss": 1.4, "auPrc": 0.85}
print(metrics)

model_eval = gapic.ModelEvaluation(
    display_name="eval",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

### 上传评估指标到模型注册表

接下来，将模型的评估指标从自定义训练作业上传到 Vertex AI 模型注册表中的相应条目。

目前，SDK 中还没有支持这种方法。相反，应该使用较低级别的 GAPIC API 接口。

In [ ]:
API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
client = gapic.ModelServiceClient(client_options={"api_endpoint": API_ENDPOINT})

client.import_model_evaluation(parent=model.resource_name, model_evaluation=model_eval)

### 列出自定义模型评估

现在您已将自定义评估指标上传到 Vertex AI 模型注册中相应的模型中，您可以使用 `list_model_evaluations()` 方法检索它。

In [ ]:
evaluation = model.list_model_evaluations()[0]
print(evaluation.gca_resource)

### 将第二个评估指标上传到模型注册表

接下来，将第二个模型评估上传到Vertex AI模型注册表中相应的条目。在这个例子中，我们将第一个评估指标称为`eval`（来自训练），将第二个评估指标称为`prod`（来自生产数据）。

In [ ]:
metrics = {"logLoss": 1.2, "auPrc": 0.87}
print(metrics)

model_prod = gapic.ModelEvaluation(
    display_name="prod",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

client.import_model_evaluation(parent=model.resource_name, model_evaluation=model_prod)

### 列出一个特定的自定义模型评估

现在您已经将第二个自定义评估指标上传到对应的模型中，您可以通过使用`display_name`进行过滤来检索特定的评估。

In [ ]:
evaluations = model.list_model_evaluations()
for evaluation in evaluations:
    if evaluation.display_name == "prod":
        print(evaluation.gca_resource)

### 上传TFHub模型的第二个版本到Vertex AI模型注册表

接下来，您将第二个版本的TFHub模型作为模型资源上传到Vertex AI模型注册表，同时添加以下附加参数：

- `parent_model`：现有模型的资源名称或模型ID，新上传的模型是该模型的一个版本。仅在上传现有模型的新版本时设置此字段。
- `is_default_version`：当设置为`True`时，新上传的模型版本将自动包含别名“default”。
- `version_aliases`：用户定义的模型版本的备用别名列表，例如`production`。
- `version_description`：模型版本的用户描述。

当在Vertex AI模型注册表中创建后续模型版本时，属性`version_id`会自动递增。在这个例子中，设置为2（第二个版本）。

In [ ]:
model_v2 = aiplatform.Model.upload(
    display_name="resnet",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    parent_model=model.resource_name,
    is_default_version=True,
    version_aliases=["v2"],
    version_description="This is the second version of the model",
)

print(model_v2)

#### 将版本2的模型的评估指标上传至模型注册表

接下来，将模型评估上传到Vertex AI模型注册表中相应的模型版本。*注意*，使用 `model_v2.resource_name` 来引用这个模型的第2个版本。

In [ ]:
metrics = {"logLoss": 1.0, "auPrc": 0.91}
print(metrics)

model_eval = gapic.ModelEvaluation(
    display_name="eval",
    metrics_schema_uri="gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    metrics=metrics,
)

client.import_model_evaluation(
    parent=model_v2.resource_name, model_evaluation=model_eval
)

最后，列出两个版本模型的评估数量。

In [ ]:
evaluations = model.list_model_evaluations()
print("Model v1 no. of evaluations", len(evaluations))
evaluations = model_v2.list_model_evaluations()
print("Model v2 no. of evaluations", len(evaluations))

## 清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
# Delete model resource
model.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False  # set True for deletion
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI